PyTorch CNN ensemble
---

Equivalent to Keras ensemble in `fit_ensemble.ipynb`


<div class="alert alert-danger" role="alert">
 <strong>Note:</strong><br>This is currently under heavy development. Nothing is gonna work in here.
</div>

In [ ]:
import torchvision
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

n_pixels = 224
n_test = 90

## Categories

In [ ]:
categories = [i for i in range(30) for j in range(12)] # creates 360 list items like so: [0, 0, 0, 0, ... 29, 29, 29, 29]

## Functions ...
... from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    """
    The train_model function handles the training and validation of a given model.
    As input, it takes a PyTorch model, a dictionary of dataloaders, a loss function,
    an optimizer, a specified number of epochs to train and validate for,
    and a boolean flag for when the model is an Inception model. The is_inception
    flag is used to accomodate the Inception v3 model, as that architecture uses an
    auxiliary output and the overall model loss respects both the auxiliary output and
    the final output, as described here. The function trains for the specified number
    of epochs and after each epoch runs a full validation step. It also keeps track of
    the best performing model (in terms of validation accuracy), and at the end of training
    returns the best performing model. 
    After each epoch, the training and validation accuracies are printed.
    """
    
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


def set_parameter_requires_grad(model, feature_extracting):
    """
    This helper function sets the .requires_grad attribute of the parameters in the model to False
    when we are feature extracting. By default, when we load a pretrained model all of the
    parameters have .requires_grad=True, which is fine if we are training from scratch or finetuning.
    However, if we are feature extracting and only want to compute gradients for the newly initialized
    layer then we want all of the other parameters to not require gradients.
    This will make more sense later.
    """
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Prepare 360 Rocks data

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "../sanders_2018/360 Rocks"

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Prepare 120 Rocks data

no train, test, validate splits ...will be later used only for testing

## Hyperparameters

In [ ]:
# Number of classes in the dataset
num_classes = 2 # TODO needs to be changed to give coordinates

# Batch size for training (change depending on how much memory you have)
batch_size = 8 

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

## Train models and save checkpoints

In [ ]:
# initiliaze the model

model = torchvision.models.resnet50(pretrained = True)
model.eval().to(device)

set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
input_size = 224

# Print the model we just instantiated
print(model_ft)

Optimizer

In [ ]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Run train and validation

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

## Load checkpoints and get predictions for validation and test sets

seems to be not sufficient / does not load to complete model as the results from an ensemble loaded this way are very poor (see also MDS notebook)

## Get MSE

## Get R²

## Save predictions to file

In [ ]:
cnn_torch_pred_file = "CNN Predictions/MDS Dimensions/cnn_torch_predicted_mds_120.txt"

np.savetxt(cnn_torch_pred_file, rocks_120_prediction)